# Import, Clean and Save Data to .csv

In this notebook, I scrape data from two subreddits, clean them by removing any deleted or removed posts and convert them in dataframes. I save the individual dataframes as raw data. I also remove any features I'm not interested in looking at and save those dataframes, as well as a concatenated dataframe with all the data both dataframes with columns of interest. 

In [1]:
import requests
import pandas as pd
import reddit_functions as rf


### Run cells below to import and save data

In [7]:
posts_front_desk = rf.get_reddit_posts('TalesFromTheFrontDesk',5000)

In [8]:
posts_front_desk.to_csv('../data/front_desk_raw.csv')

In [2]:
posts_tech_support = rf.get_reddit_posts('talesfromtechsupport',5000)

In [6]:
posts_tech_support.to_csv('../data/tech_support_raw.csv')

In [9]:
set(posts_front_desk.columns).intersection(set(posts_tech_support.columns))

{'all_awardings',
 'allow_live_comments',
 'author',
 'author_cakeday',
 'author_flair_background_color',
 'author_flair_css_class',
 'author_flair_richtext',
 'author_flair_template_id',
 'author_flair_text',
 'author_flair_text_color',
 'author_flair_type',
 'author_fullname',
 'author_is_blocked',
 'author_patreon_flair',
 'author_premium',
 'awarders',
 'banned_by',
 'can_mod_post',
 'contest_mode',
 'created_utc',
 'distinguished',
 'domain',
 'edited',
 'full_link',
 'gilded',
 'gildings',
 'id',
 'is_created_from_ads_ui',
 'is_crosspostable',
 'is_meta',
 'is_original_content',
 'is_reddit_media_domain',
 'is_robot_indexable',
 'is_self',
 'is_video',
 'link_flair_background_color',
 'link_flair_css_class',
 'link_flair_richtext',
 'link_flair_text',
 'link_flair_text_color',
 'link_flair_type',
 'locked',
 'media_only',
 'no_follow',
 'num_comments',
 'num_crossposts',
 'over_18',
 'parent_whitelist_status',
 'permalink',
 'pinned',
 'post_hint',
 'preview',
 'pwls',
 'removed_

In [10]:
columns_to_save = ['subreddit', 'title', 'selftext', 'created_utc', 'num_comments']

In [11]:
red_df = pd.concat([posts_front_desk[columns_to_save],
                    posts_tech_support[columns_to_save]])


### Run cells below to check for missing data and pull additional data if necessary

In [ ]:
red_df = pd.read_csv('../data/all_posts_text_title.csv')

In [13]:
red_df.isna().sum()

subreddit        0
title            0
selftext        70
created_utc      0
num_comments     0
dtype: int64

In [14]:
# check which subreddit the missing data is from

red_df[red_df['selftext'].isna()]['subreddit'].value_counts()

talesfromtechsupport     49
TalesFromTheFrontDesk    21
Name: subreddit, dtype: int64

In [15]:
# it's not a lot of data so I will drop the rows with missing post text
red_df.dropna(inplace=True)

In [16]:
red_df.dtypes

subreddit       object
title           object
selftext        object
created_utc      int64
num_comments     int64
dtype: object

In [ ]:
# drop unnamed column
red_df.drop(columns=['Unnamed: 0'], inplace=True)

In [17]:
# create a word count column to look for any deleted posts
red_df['post_length'] = [len(post.strip().split(' ')) for post in red_df['selftext']]
red_df.head()

,subreddit,title,selftext,created_utc,num_comments,post_length
0,TalesFromTheFrontDesk,Update from my post about my coworker,Here is the original [post](https://www.reddit...,1648499626,0,189
1,TalesFromTheFrontDesk,You’re forcing me!,"So, for context, I work at a 3-star-ish busine...",1648471124,0,501
3,TalesFromTheFrontDesk,“I actually have to read what I’m signing for?”,So this literally happened as I’m walking in a...,1648445630,0,305
5,TalesFromTheFrontDesk,The Straw(s) That Broke the Camel's Back,"Hey guys, it's been a while since I've posted....",1648441064,0,1185
6,TalesFromTheFrontDesk,DM Report Ideas?,"Hello Front Desk,\n\nApologies if this is agai...",1648435782,0,179


In [26]:
# check any posts that just have a single word
red_df[red_df['post_length']==1]['selftext'].value_counts()

                                                                75
[deleted]                                                       31
[https://youtu.be/mWINe-m0hsk](https://youtu.be/mWINe-m0hsk)     1
[https://imgur.com/RfPFAcB](https://imgur.com/RfPFAcB)           1
Name: selftext, dtype: int64

In [32]:
# which subredit do they primarily come from
red_df[red_df['post_length']==1]['subreddit'].value_counts()

Series([], Name: subreddit, dtype: int64)

In [33]:
# remove any posts that have just one word
red_df = red_df[red_df['post_length']>1]

In [34]:
# double check counts of posts from the two subreddits to make sure they're similar
red_df['subreddit'].value_counts()

TalesFromTheFrontDesk    4999
talesfromtechsupport     4886
Name: subreddit, dtype: int64

In [36]:
red_df.to_csv('../data/all_posts_cleaned.csv', index=False)